# Lung Cancer Survival Prediction Endpoint Demo

In this endpoint demo notebook, we demonstrate how to send inference requests to an pre-deployed endpoint and get the model response.

To find more details of an end-to-end solution for data processing, feature store, model training and deployement using SageMaker, check out the solution notebook `xxxxxxx.ipynb`. It shows how-to for the following steps: 1/ processing multi-modal data (genomic, clinical, medical imaging) to obtain ML features, 2/ ingesting and managing multi-modal features in SageMaker Feature Store, 3/ training a survival status prediction model using PCA and XGBoost, 4/ hosting a model for inference. The exposition in this notebook is deliberately brief. 

>**<span style="color:RED">Important</span>**: 
>This solution is for demonstrative purposes only. It is not for clinical use. The ML inference should not be used to inform any clinical decision. The associated notebooks, including the trained model and sample data, are not intended for production.

### Step 1: Read in the solution config

In [ ]:
import json

SOLUTION_CONFIG = json.load(open("stack_outputs.json"))
ROLE = SOLUTION_CONFIG["IamRole"]
SOLUTION_BUCKET = SOLUTION_CONFIG["SolutionS3Bucket"]
REGION = SOLUTION_CONFIG["AWSRegion"]
SOLUTION_NAME = SOLUTION_CONFIG["SolutionName"]
BUCKET = SOLUTION_CONFIG["S3Bucket"]

### Step 2: Download and read in the multimodal dataset for inference

The test multimodal dataset consists of genomic, clinical and imaging features for X patients. The features have been condensed by PCA from 216 to 65 principal components.

In [ ]:
from sagemaker.s3 import S3Downloader

input_data_bucket = f"s3://{SOLUTION_BUCKET}-{REGION}/{SOLUTION_NAME}/data"
print("original data: ")
S3Downloader.list(input_data_bucket)

#### Download the data for inference from S3

In [ ]:
inference_data = f"{input_data_bucket}/inference_data.csv"
!aws s3 cp $inference_data .

In [ ]:
import pandas as pd

df = pd.read_csv("inference_data.csv")
print(df.shape)
df.head()

The features are principal components computed from 216 features. The original feature vector include features from genomic secondary analysis, clinical health records, and radiomic features from within the lung tumor in computed tomography images. 

#### Snapshot of data
##### Clinical
![clinical-data](../images/clinical-data-screenshot.png)

#### Genomic
![genomic-data](../images/genomic-secondary.png)

#### Medical imaging
![imaging-data](../images/CT-tumor-overlay.png)

### Step 3: Predicting survival status

If you want to use the demo endpoint successfully, your dataframe columns should be identical to the `df_tabtext_score` as shown in the previous step.

In [ ]:
import sagemaker
from sagemaker import Predictor
import numpy as np

endpoint_name = SOLUTION_CONFIG["SolutionPrefix"] + "-demo-endpoint" 


predictor = Predictor(
    endpoint_name = endpoint_name,
    sagemaker_session = sagemaker.Session(),
    deserializer =  sagemaker.deserializers.JSONDeserializer(),
    serializer = sagemaker.serializers.CSVSerializer(),
)

prediction = predictor.predict(df.values)


Let's take a look at the total count of predicted survival status.

In [ ]:
print(np.bincount(prediction))